In [1]:
!nvidia-smi

Tue May 10 20:43:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 4.0 MB 4.4 MB/s 
     |████████████████████████████████| 77 kB 3.0 MB/s 
     |████████████████████████████████| 880 kB 57.1 MB/s 
     |████████████████████████████████| 596 kB 41.1 MB/s 
     |████████████████████████████████| 6.6 MB 66.6 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=fc00daef9c9a7b9658ac6dd60d72b34fbf9b5d434e80e062d96df95a87f48430
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 342 kB 8.1 MB/s 
     |████████████████████████████████| 1.1 MB 67.4 MB/s 
     |████████████████████████████████| 136 kB 79.7 MB/s 
     |████████████████████████████████| 212 kB 70.1 MB/s 
     |██████████████████████████████

In [3]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

In [4]:
from google.colab import drive
from datasets import load_dataset
from transformers import AutoTokenizer
from datasets import ClassLabel
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
import numpy as np
import pandas as pd
from datasets import load_metric
from transformers import Trainer
from sklearn.metrics import classification_report

drive.mount('/content/drive')
google_path = 'drive/MyDrive/'

Mounted at /content/drive


In [5]:
all_langs = [
    'ru',
    'uk',
    'ka',
    'he',
    'en',
    'de',
    'be',
    'kk',
    'az',
    'hy',
]

labels = ClassLabel(num_classes=10, names=all_langs)

In [6]:
dataset = load_dataset('csv', 
                       data_files={'train': google_path+"ml_train.csv", 'test': google_path+"ml_val.csv"},
                       delimiter='\t'
                       )
dataset

Using custom data configuration default-bdb1ae0925d8be5d


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-bdb1ae0925d8be5d/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'ru', 'uk', 'ka', 'he', 'en', 'de', 'be', 'kk', 'az', 'hy'],
        num_rows: 89152
    })
    test: Dataset({
        features: ['text', 'ru', 'uk', 'ka', 'he', 'en', 'de', 'be', 'kk', 'az', 'hy'],
        num_rows: 9906
    })
})

In [7]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

def tokenize(batch):
    encoding = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)
    labels_batch = {k: batch[k] for k in batch.keys() if k in all_langs}
    labels_matrix = np.zeros((len(batch["text"]), len(all_langs)))
    for idx, label in enumerate(all_langs):
        labels_matrix[:, idx] = labels_batch[label]

    encoding["labels"] = labels_matrix.tolist()
    return encoding

tokenized_datasets = dataset.map(tokenize, batched=True)

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

  0%|          | 0/90 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [8]:
train_dataset = tokenized_datasets["train"].shuffle(seed=42)
eval_dataset = tokenized_datasets["test"].shuffle(seed=42)

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(
    "xlm-roberta-base",
     num_labels=10, 
     problem_type="multi_label_classification", 
)
model.config.id2label = {id: lang for id, lang in enumerate(all_langs)}
model.config.label2id = {lang: id for id, lang in enumerate(all_langs)}

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [27]:
batch_size = 16

training_args = TrainingArguments(
    output_dir=google_path+"ml-lang-xlm-roberta-base",
    # overwrite_output_dir=True,
    # logging_strategy = "epoch",
    # save_strategy = "epoch",
    evaluation_strategy = "epoch",
    # evaluation_strategy = "steps",
    # save_steps=5000,
    # eval_steps=10,
    # save_total_limit=2,
    # num_train_epochs=2,
    # learning_rate=2e-5,
    # weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
)

In [28]:
def multi_label_metrics(predictions, labels, threshold=0.5):
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    report = classification_report(y_pred, y_true, target_names=all_langs, output_dict=True)
    scores = {f"{lang}_{metric}": score for lang, lang_report in report.items() for metric, score in lang_report.items()}
    metrics = {
        'f1': f1_micro_average,
        'roc_auc': roc_auc,
        'accuracy': accuracy,
    }
    return {**metrics, **scores}

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(predictions=preds, labels=p.label_ids)
    return result

In [29]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: uk, ru, ka, de, hy, text, az, en, he, kk, be. If uk, ru, ka, de, hy, text, az, en, he, kk, be are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 89152
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 11144


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy,Ru Precision,Ru Recall,Ru F1-score,Ru Support,Uk Precision,Uk Recall,Uk F1-score,Uk Support,Ka Precision,Ka Recall,Ka F1-score,Ka Support,He Precision,He Recall,He F1-score,He Support,En Precision,En Recall,En F1-score,En Support,De Precision,De Recall,De F1-score,De Support,Be Precision,Be Recall,Be F1-score,Be Support,Kk Precision,Kk Recall,Kk F1-score,Kk Support,Az Precision,Az Recall,Az F1-score,Az Support,Hy Precision,Hy Recall,Hy F1-score,Hy Support,Micro avg Precision,Micro avg Recall,Micro avg F1-score,Micro avg Support,Macro avg Precision,Macro avg Recall,Macro avg F1-score,Macro avg Support,Weighted avg Precision,Weighted avg Recall,Weighted avg F1-score,Weighted avg Support,Samples avg Precision,Samples avg Recall,Samples avg F1-score,Samples avg Support
1,0.161500,0.110949,0.947844,0.953939,0.762063,0.936627,0.954845,0.945648,2879,0.933972,0.958904,0.946274,2847,0.866415,1.000000,0.928427,2523,0.882789,0.999619,0.937578,2622,0.874788,0.980601,0.924677,2629,0.875042,0.982463,0.925646,2680,0.941071,0.985327,0.962691,2658,0.935530,0.998471,0.965976,2616,0.952230,0.991950,0.971685,2733,0.945725,0.998128,0.971220,2671,0.913810,0.984511,0.947844,26858,0.914419,0.985031,0.947982,26858,0.915121,0.984511,0.948119,26858,0.925168,0.983859,0.946365,26858


The following columns in the evaluation set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: uk, ru, ka, de, hy, text, az, en, he, kk, be. If uk, ru, ka, de, hy, text, az, en, he, kk, be are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9906
  Batch size = 16
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to drive/MyDrive/ml-lang-xlm-roberta-base/checkpoint-5572
Configuration saved in drive/MyDrive/ml-lang-xlm-roberta-base/checkpoint-5572/config.json
Model weights saved in drive/MyDrive/ml-lang-xlm-roberta-base/checkpoint-5572/pytorch_model.bin
The fol

In [17]:
trainer.save_model()

Saving model checkpoint to drive/MyDrive/ml-lang-xlm-roberta-base
Configuration saved in drive/MyDrive/ml-lang-xlm-roberta-base/config.json
Model weights saved in drive/MyDrive/ml-lang-xlm-roberta-base/pytorch_model.bin


In [15]:
model_path = 'drive/MyDrive/ml-lang-xlm-roberta-base'
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [17]:
model.config.id2label = {id: lang for id, lang in enumerate(all_langs)}
model.config.label2id = {lang: id for id, lang in enumerate(all_langs)}

In [18]:
model.config

XLMRobertaConfig {
  "_name_or_path": "drive/MyDrive/ml-lang-xlm-roberta-base",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "ru",
    "1": "uk",
    "2": "ka",
    "3": "he",
    "4": "en",
    "5": "de",
    "6": "be",
    "7": "kk",
    "8": "az",
    "9": "hy"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "az": 8,
    "be": 6,
    "de": 5,
    "en": 4,
    "he": 3,
    "hy": 9,
    "ka": 2,
    "kk": 7,
    "ru": 0,
    "uk": 1
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "multi_label_classification",
  "tor

In [32]:
preds = trainer.predict(eval_dataset)
preds = preds.predictions

The following columns in the test set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: he, text, uk, en, ka, kk, hy, be, az, de, ru. If he, text, uk, en, ka, kk, hy, be, az, de, ru are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 9906
  Batch size = 16


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [49]:
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(torch.Tensor(preds))
probs

tensor([[0.9976, 0.0113, 0.0602,  ..., 0.0392, 0.0769, 0.9977],
        [0.7149, 0.0705, 0.0783,  ..., 0.0460, 0.9995, 0.0232],
        [0.9826, 0.0191, 0.9967,  ..., 0.0125, 0.9987, 0.0108],
        ...,
        [0.0100, 0.9966, 0.0565,  ..., 0.0212, 0.9942, 0.0138],
        [0.0047, 0.0048, 0.0232,  ..., 0.0070, 0.0074, 0.0080],
        [0.0117, 0.0125, 0.9928,  ..., 0.9895, 0.0143, 0.0074]])

In [50]:
probs[probs > 0.5] = 1
probs[probs <= 0.5] = 0
probs

tensor([[1., 0., 0.,  ..., 0., 0., 1.],
        [1., 0., 0.,  ..., 0., 1., 0.],
        [1., 0., 1.,  ..., 0., 1., 0.],
        ...,
        [0., 1., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 1., 0., 0.]])

In [55]:
eval_dataset[4]['text']

'Мақсаты: 1.Құқық сөзінің мән – мағынасымен, оның түрлерімен таныстыра отырып, Бала құқығын қорғау Конвенциясының қабылдану ерекшеліктері мен өмірдегі атқаратын ролі туралы түсіндіру.\n2. Бала құқығын қорғау Конвенциясының оқушы өміріндегі алатын орны туралы оқушылардың өзіндік ойлау қабілетін дамытып, шығармашылыққа баулу.\n(Топ жетекшілері тапсырмалар'

In [52]:
probs = probs.int().tolist()
pd.DataFrame(data=probs, columns=all_langs)

,ru,uk,ka,he,en,de,be,kk,az,hy
0,1,0,0,0,1,0,1,0,0,1
1,1,0,0,0,0,0,1,0,1,0
2,1,0,1,0,0,0,0,0,1,0
3,0,0,0,0,0,1,0,1,0,0
4,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
9901,0,0,0,0,1,0,0,0,0,0
9902,1,1,0,0,0,0,0,0,1,0
9903,0,1,0,0,1,0,0,0,1,0
9904,0,0,0,1,0,0,0,0,0,0


In [28]:
sigmoid = torch.nn.Sigmoid()

preds = model(**tokenizer(input, return_tensors='pt')).logits
probs = sigmoid(preds)
probs = probs.detach()[0]
for ind, prob in enumerate(probs):
    if prob > 0.5:
        print(labels.int2str(ind))

ru
uk
